In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import time
import os 
from thop import profile
from torchsummary import summary
from tqdm import tqdm
import torch

import numpy as np
import pandas as pd

In [13]:
batch_size = 100

random_seed = 42
g = torch.Generator()
g.manual_seed(random_seed)

train_dataset=pd.read_csv(os.getcwd()+'/dataset/train.csv')
test_dataset=pd.read_csv(os.getcwd()+'/dataset/test.csv')

print(train_dataset.head(5))
print(test_dataset.head(5))

   id  Gender        Age    Height      Weight family_history_with_overweight  \
0   0    Male  24.443011  1.699998   81.669950                            yes   
1   1  Female  18.000000  1.560000   57.000000                            yes   
2   2  Female  18.000000  1.711460   50.165754                            yes   
3   3  Female  20.952737  1.710730  131.274851                            yes   
4   4    Male  31.641081  1.914186   93.798055                            yes   

  FAVC      FCVC       NCP        CAEC SMOKE      CH2O SCC       FAF  \
0  yes  2.000000  2.983297   Sometimes    no  2.763573  no  0.000000   
1  yes  2.000000  3.000000  Frequently    no  2.000000  no  1.000000   
2  yes  1.880534  1.411685   Sometimes    no  1.910378  no  0.866045   
3  yes  3.000000  3.000000   Sometimes    no  1.674061  no  1.467863   
4  yes  2.679664  1.971472   Sometimes    no  1.979848  no  1.967973   

        TUE       CALC                 MTRANS           NObeyesdad  
0  0.976473

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,generator=g)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [11]:
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
print(f"Label: {label}")

KeyError: 1454

In [ ]:
print('Start: ', time.time())

train_time = 0

# Training loop
for epoch in range(num_epochs):
    # Training phase
    #break;

    start_train = time.time()

    train_correct = 0
    train_total = 0
    train_loss = 0
    # Sets the model in training mode.
    model = model.train()
    for batch_idx, (images, labels) in enumerate(tqdm(train_loader)):
        # TODO: Put the images and labels on the GPU
        images = images.to(torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
        labels = labels.to(torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
        # Sets the gradients to zero
        optimizer.zero_grad()
        # The actual inference
        outputs = model(images)
        # Compute the loss between the predictions (outputs) and the ground-truth labels
        loss = criterion(outputs, labels)
        # Do backpropagation to update the parameters of your model
        loss.backward()
        # Performs a single optimization step (parameter update)
        optimizer.step()
        train_loss += loss.item()
        # The outputs are one-hot labels, we need to find the actual predicted
        # labels which have the highest output confidence
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
        # Print every 100 steps the following information
        if (batch_idx + 1) % 100 == 0 or (batch_idx == 389):
            print('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f Acc: %.2f%%' % (epoch + 1, num_epochs, batch_idx + 1,
                                                                             len(train_dataset) // batch_size,
                                                                             train_loss / (batch_idx + 1),
                                                                             100. * train_correct / train_total))
    
    train_time = train_time + time.time() - start_train

    # Testing phase
    test_correct = 0
    test_total = 0
    test_loss = 0
    # Sets the model in evaluation mode
    model = model.eval()
    # Disabling gradient calculation is useful for inference.
    # It will reduce memory consumption for computations.
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(tqdm(test_loader)):
            # TODO: Put the images and labels on the GPU

            images = images.to(torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
            labels = labels.to(torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
            # Perform the actual inference
            outputs = model(images)
            # Compute the loss
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            # The outputs are one-hot labels, we need to find the actual predicted
            # labels which have the highest output confidence
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()
    print('Test loss: %.4f Test accuracy: %.2f %%' % (test_loss / (batch_idx + 1),100. * test_correct / test_total))

print('End Time: ', time.time())

print('\nTotal training time: ', train_time)

sample_input = torch.randn(1,3,32,32)
sample_input = sample_input.to(torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

macs_thop, params_thop = profile(model, inputs=(sample_input, ))
print("\n\n")

print("Macs from thop.profile: ", macs_thop)
print("Params from thop.profile: ", params_thop)

print("\n\n")

torchsum_out = summary(model, (3,32,32))

print(torchsum_out)